In [2]:
import numpy as np
import pandas as pd
import scipy as sp
#import matplotlib.pyplot as plt
from scipy import stats
import os
import time
import itertools
from collections import Counter
from os.path import isfile, join
from random import randint, shuffle
import warnings
#import keras
#from keras.models import Sequential
#from keras.layers import Dense, Dropout, Activation
#from keras.optimizers import SGD
#from keras.callbacks import ModelCheckpoint
import sklearn
from sklearn.preprocessing import StandardScaler
import gc
from joblib import Parallel, delayed 
from multiprocessing import *
import sys, os
#sys.path.append(os.path.dirname(os.path.expanduser('../../pers_not.py')))
#from pers_not import *
#send_personal_notification('{} finished :sunglasses:'.format(sys.argv[0]))
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

warnings.filterwarnings("ignore")

In [ ]:
# fare le cose per TCA

In [15]:
def rabalta_true(df1):
    df = df1.copy()
    df = df.stack().to_frame().reset_index()
    df.columns = ['system', 'topic', 'AP']
    

    cols_top = df['system'].values
    cols_top = [x.replace('_', '.') for x in cols_top]
    cols_meth = df['topic'].values

    col_idx_arr = list(zip(cols_top, cols_meth))
    col_idx = pd.MultiIndex.from_tuples(col_idx_arr)
    df.index = col_idx

    df.index = [df.index.get_level_values(
        0), df.index.map('{0[0]}_{0[1]}'.format)]
    df.reset_index(inplace=True)
    df.drop('level_0', 1, inplace=True)
    df.rename(columns={'level_1': 'IND'}, inplace=True)
    df.set_index('IND', inplace=True)
    return df

In [40]:
collections = ['TREC3', 'TREC5', 'TREC6', 'TREC7', 'AH99', 'TREC2001',
               #'R04', 'TB04', 'R05', 'TB05', 'TB06', 'WEB11', 'WEB12', 'WEB13', 'WEB14gd'
              ]


arg_instances = []
for collection_train in collections:
    for collection_eval in collections:
        #print('='*50)
        #print('{}-{}'.format(collection_train, others_coll))

        train = pd.read_pickle('./_pickles/TCA-notind/train{}-eval{}-train.pickle'.format(collection_train, collection_eval))
        test = pd.read_pickle('./_pickles/TCA-notind/train{}-eval{}-test.pickle'.format(collection_train, collection_eval))
        
        train_ind = pd.read_csv('../src/Tables/{}.csv'.format(collection_train), index_col=0)
        test_ind = pd.read_csv('../src/Tables/{}.csv'.format(collection_eval), index_col=0)
        
        train_ind = rabalta_true(train_ind)
        test_ind = rabalta_true(test_ind)
        
        #display(train_ind.head())
        #display(test_ind.head())
        
        train.index = train_ind.index
        test.index = test_ind.index
        
        train.to_pickle('./_pickles/TCA/train{}-eval{}-train.pickle'.format(collection_train, collection_eval))
        test.to_pickle('./_pickles/TCA/train{}-eval{}-test.pickle'.format(collection_train, collection_eval))
        
        #print('\tdone on {}-{}'.format(collection_eval, others_coll))
        #assert False
print('done.')

done.


# Metodi per processare i dati di input

In [2]:
def get_features(df1, feature):
    df = df1.copy()
    cols = [x for x in df.columns.values if x.split('_')[1] == feature]
    df = df[cols]
    return df


def apply_filter(filter_, data_):
    filter_.inputformat(data_)
    return filter_.filter(data_)


def rabalta_feature(df1):
    df = df1.copy()
    cols_top = [x.split('_')[0] for x in df.columns.values]
    cols_meth = [x.split('_')[1] for x in df.columns.values]

    col_idx_arr = list(zip(cols_meth, cols_top))
    col_idx = pd.MultiIndex.from_tuples(col_idx_arr)
    df.columns = col_idx

    rab = df.stack()
    rab.index = [rab.index.get_level_values(
        0), rab.index.map('{0[0]}_{0[1]}'.format)]
    rab.reset_index(inplace=True)

    rab.drop(rab.columns[[0]], axis=1, inplace=True)

    rab.rename(columns={'level_1': 'IND'}, inplace=True)
    rab.set_index('IND', inplace=True)

    rab['system'] = [x.split('_')[0] for x in rab.index.values]
    rab['system'] = [x.replace('_', '.') for x in rab['system']]
    rab['topic'] = [x.split('_')[1] for x in rab.index.values]

    return rab


def rabalta_true(df1):
    """
    Questo metodo produce una nuova tabella a partire dal dataframe di input,
    definita dalle colonne 'IND', system', 'topic' e 'AP'.
    """
    df = df1.copy()
    df = df.stack().to_frame().reset_index()
    df.columns = ['system', 'topic', 'AP']

    cols_top = df['system'].values
    cols_top = [x.replace('_', '.') for x in cols_top]
    cols_meth = df['topic'].values

    col_idx_arr = list(zip(cols_top, cols_meth))
    col_idx = pd.MultiIndex.from_tuples(col_idx_arr)
    df.index = col_idx

    df.index = [df.index.get_level_values(
        0), df.index.map('{0[0]}_{0[1]}'.format)]
    df.reset_index(inplace=True)
    df.drop('level_0', 1, inplace=True)
    df.rename(columns={'level_1': 'IND'}, inplace=True)
    df.set_index('IND', inplace=True)
    return df


def move_column(df1, coltomove, position):
    """
    :param df1: data frame
    :param coltomove: colonna da spostare
    :param position: posizione in cui collocare la colonna
    :return: data frame aggiornato
    """
    df = df1.copy()
    cols = list(df)
    cols.insert(position, cols.pop(cols.index(coltomove)))
    df = df.ix[:, cols]
    return df

# Dataset

In [4]:
collections = ['TREC3', 'TREC5', 'TREC6', 'TREC7', 'AH99', 'TREC2001',
               'R04', 'TB04', 'R05', 'TB05', 'TB06', 'WEB11', 'WEB12', 'WEB13', 'WEB14gd']
collectionsy = ['1994', '1996', '1997', '1998', '1999', '2001', '2004',
                '2004', '2005', '2005', '2006', '2011', '2012', '2013', '2014']
rev_collections = list(reversed(collections))

collections_DATA = pd.DataFrame(columns=['collection', 'year'])
collections_DATA['collection'] = collections
collections_DATA['year'] = collectionsy

collections_DATA['collection'] = collections_DATA['collection'].astype(str)
collections_DATA['year'] = collections_DATA['year'].astype(int)

# Machine Learning


In [5]:
def prepara(collection, s_tablepath, s_dataframesmlpath):

    truemat = pd.read_csv(s_tablepath + 'Tables/' +
                          collection + '.csv', index_col=0)
    # truemat = truemat.sub(truemat.mean(axis=0), axis=1)
    truemat.sort_index(inplace=True)
    truemat = rabalta_true(truemat)
    m_nodiv = s_dataframesmlpath + '{}_Features.pickle'.format(collection)
    m_nodiv = pd.read_pickle(m_nodiv)

    feats = np.unique([x.split('_')[1] for x in m_nodiv.columns])

    m_nodiv_norm = pd.DataFrame()

    for feat in feats:
        giuste = [x for x in m_nodiv.columns if x.split('_')[1] == feat]
        subs = m_nodiv[giuste]
        # subs = subs.sub(subs.mean(axis=0), axis=1)
        m_nodiv_norm = pd.concat([m_nodiv_norm, subs], 1)

    m_nodiv = rabalta_feature(m_nodiv_norm)
    systems = np.unique(m_nodiv['system'])
    merged = pd.merge(truemat[['AP']], m_nodiv,
                      left_index=True, right_index=True)
    merged = move_column(merged, 'AP', len(merged.columns.values))
    
    merged.drop(['system', 'topic'], 1, inplace=True)

    return merged

In [11]:

def fun(args):
    collection_eval, others_coll = args
    
    rev_collections = list(reversed(collections))

    s_tablepath = '../src/'
    s_dataframesmlpath = '../src/features/'
    improve_kind = 'base'
    
    df_result = pd.DataFrame(
    columns=['train_on', 'eval_on', 'improve_kind', 'kind', 'df'])

    m_test = prepara(collection_eval, s_tablepath, s_dataframesmlpath)
    for i_others, collection in enumerate(others_coll):
        au = prepara(collection, s_tablepath, s_dataframesmlpath)
        if i_others == 0:
            m_train = pd.DataFrame(columns=au.columns.values)
            m_train = pd.concat([m_train, au])
        else:
            m_train = pd.concat([m_train, au])

    m_train = move_column(m_train, 'AP', len(m_train.columns.values))
    col_order = m_train.columns
    m_test = m_test[col_order]
    m_train = m_train.fillna(0.0)
    m_test = m_test.fillna(0.0)

    ind_res = m_test.index.values

    display(m_train.head())
    display(m_test.head())
    #assert False
    
    
    ###########################
    
    train_values = m_train.values
    last_train_column_index = train_values[0, :].size - 1
    x_train = train_values[:, 0:last_train_column_index]
    y_train = train_values[:, last_train_column_index]

    test_values = m_test.values
    last_test_column_index = test_values[0, :].size - 1
    x_test = test_values[:, 0:last_test_column_index]
    y_test = test_values[:, last_test_column_index]
    
    ####################################
    
    import keras.backend as K
    import tensorflow as tf

    
    def customLoss(yTrue,yPred):
        x = yTrue
        y = yPred
        
        mx = K.mean(x)
        my = K.mean(y)
        xm, ym = x-mx, y-my
        r_num = K.sum(tf.multiply(xm,ym))
        r_den = K.sqrt(tf.multiply(K.sum(K.square(xm)), K.sum(K.square(ym))))
        r = r_num / r_den

        r = K.maximum(K.minimum(r, 1.0), -1.0)
        return  1- K.square(r)
    
    
    model = Sequential()
    model.add(Dense(output_dim = 16, init = 'uniform', activation = 'relu', input_dim = x_train.shape[1]))
    model.add(Dense(output_dim = 1, kernel_initializer='normal'))
    
    model.compile(loss=customLoss, optimizer='adam', metrics=['accuracy'])
    
    scale = StandardScaler()
    x_train = scale.fit_transform(x_train)
    x_test = scale.transform(x_test)

    fitted_model = model.fit(x_train, y_train, epochs=10, verbose = 0)
    

    predicted = model.predict(x_test)
    predicted = [item for sublist in predicted for item in sublist]
    #predicted = [int(x>0.5) for x in predicted]
    
    #print(predicted)
    #print(y_test)
    print('eval-{} -- pearson: {}'.format(collection_eval, sp.stats.pearsonr(predicted, y_test)[0]  )   )
    
    predicted = model.predict(x_test)

    df_res = pd.DataFrame(index=ind_res)
    df_res['actual'] = y_test
    df_res['predicted'] = predicted
    df_res['system'] = [x.split('_')[0] for x in df_res.index.values]
    df_res['topic'] = [x.split('_')[1] for x in df_res.index.values]

    df_result.loc[len(df_result)] = [len(others_coll),
                                     collection_eval, 'Base', 'DNN', df_res]
    
    df_result.to_pickle('./pickles/train{}-eval{}-alg{}.pickle'.format(len(others_coll), 
                                                                       collection_eval,
                                                                       'DNN'))
    
    gc.collect()
    return 1
    
    
#fun(  ('TREC6', ['TREC3', 'TREC5'])   )

In [ ]:
## not tl

In [12]:
arg_instances = []

for i_coll, collection_eval in enumerate(rev_collections[:-1]):
    others_coll = list(collections_DATA[collections_DATA['year'] < int(
        collections_DATA[collections_DATA['collection'] == collection_eval]['year'].values)]['collection'].values)
    
    arg_instances.append(   (collection_eval,others_coll)    )
    
    
print(arg_instances)

[('WEB14gd', ['TREC3', 'TREC5', 'TREC6', 'TREC7', 'AH99', 'TREC2001', 'R04', 'TB04', 'R05', 'TB05', 'TB06', 'WEB11', 'WEB12', 'WEB13']), ('WEB13', ['TREC3', 'TREC5', 'TREC6', 'TREC7', 'AH99', 'TREC2001', 'R04', 'TB04', 'R05', 'TB05', 'TB06', 'WEB11', 'WEB12']), ('WEB12', ['TREC3', 'TREC5', 'TREC6', 'TREC7', 'AH99', 'TREC2001', 'R04', 'TB04', 'R05', 'TB05', 'TB06', 'WEB11']), ('WEB11', ['TREC3', 'TREC5', 'TREC6', 'TREC7', 'AH99', 'TREC2001', 'R04', 'TB04', 'R05', 'TB05', 'TB06']), ('TB06', ['TREC3', 'TREC5', 'TREC6', 'TREC7', 'AH99', 'TREC2001', 'R04', 'TB04', 'R05', 'TB05']), ('TB05', ['TREC3', 'TREC5', 'TREC6', 'TREC7', 'AH99', 'TREC2001', 'R04', 'TB04']), ('R05', ['TREC3', 'TREC5', 'TREC6', 'TREC7', 'AH99', 'TREC2001', 'R04', 'TB04']), ('TB04', ['TREC3', 'TREC5', 'TREC6', 'TREC7', 'AH99', 'TREC2001']), ('R04', ['TREC3', 'TREC5', 'TREC6', 'TREC7', 'AH99', 'TREC2001']), ('TREC2001', ['TREC3', 'TREC5', 'TREC6', 'TREC7', 'AH99']), ('AH99', ['TREC3', 'TREC5', 'TREC6', 'TREC7']), ('TREC7',

In [ ]:
## TL

In [22]:
arg_instances = []

for collection_eval in collections:
    for others_coll in collections:
        arg_instances.append(   (collection_eval, [others_coll])    )
        arg_instances.append(   (others_coll, [collection_eval])    )
    
print(arg_instances)

[('TREC3', []), ('TREC5', ['TREC3']), ('TREC6', ['TREC3', 'TREC5']), ('TREC7', ['TREC3', 'TREC5', 'TREC6']), ('AH99', ['TREC3', 'TREC5', 'TREC6', 'TREC7']), ('TREC2001', ['TREC3', 'TREC5', 'TREC6', 'TREC7', 'AH99']), ('R04', ['TREC3', 'TREC5', 'TREC6', 'TREC7', 'AH99', 'TREC2001']), ('TB04', ['TREC3', 'TREC5', 'TREC6', 'TREC7', 'AH99', 'TREC2001']), ('R05', ['TREC3', 'TREC5', 'TREC6', 'TREC7', 'AH99', 'TREC2001', 'R04', 'TB04']), ('TB05', ['TREC3', 'TREC5', 'TREC6', 'TREC7', 'AH99', 'TREC2001', 'R04', 'TB04']), ('TB06', ['TREC3', 'TREC5', 'TREC6', 'TREC7', 'AH99', 'TREC2001', 'R04', 'TB04', 'R05', 'TB05']), ('WEB11', ['TREC3', 'TREC5', 'TREC6', 'TREC7', 'AH99', 'TREC2001', 'R04', 'TB04', 'R05', 'TB05', 'TB06']), ('WEB12', ['TREC3', 'TREC5', 'TREC6', 'TREC7', 'AH99', 'TREC2001', 'R04', 'TB04', 'R05', 'TB05', 'TB06', 'WEB11']), ('WEB13', ['TREC3', 'TREC5', 'TREC6', 'TREC7', 'AH99', 'TREC2001', 'R04', 'TB04', 'R05', 'TB05', 'TB06', 'WEB11', 'WEB12']), ('WEB14gd', ['TREC3', 'TREC5', 'TREC6

In [ ]:
## eseguo

In [13]:
results = Parallel(n_jobs=1,verbose=5)(delayed(fun)(arg) for arg in arg_instances)
send_personal_notification('{} finished :sunglasses:'.format(sys.argv[0]))
print('done.')

,ALLFIVE,ASLAM,NurayBiasBorda30,NurayBiasCondorcet30,NurayBiasRankPosition30,NurayNormalBorda30,NurayNormalCondorcet30,NurayNormalRankPosition30,SAKAI30,SINGLE,SINGLEMINUSALLFIVE,SOBOROFF,WUCBASIC,WUCV1,WUCV2,WUCV3,WUCV4,AP
acqnt1_190,0.62,0.135593,0.0098,0.0006,0.0094,0.0297,0.0252,0.0431,0.0116,-86.48,-85.86,0.0088,1965,2117722,2069.97,2254022.56,19.657637,0.0043
acqnt1_191,8.22,0.459965,0.0317,0.0247,0.0317,0.0215,0.0581,0.0726,0.0248,-78.28,-70.06,0.0219,5347,6039575,4924.62,5566779.21,-49.989827,0.0229
acqnt1_192,20.40,5.964804,0.3761,0.3579,0.3761,0.3294,0.3129,0.0317,0.4448,-71.80,-51.40,0.0178,8845,10141396,8519.05,9862933.30,-30.031752,0.1330
acqnt1_193,8.64,5.961513,0.1683,0.2530,0.1683,0.1165,0.1410,0.0711,0.1642,-84.84,-76.20,0.0225,4183,5196204,4430.26,5446983.46,18.320037,0.1135
acqnt1_194,1.16,0.625675,0.0158,0.0161,0.0140,0.0705,0.0358,0.0218,0.0225,-88.64,-87.48,0.0144,2543,2800281,2844.00,3148700.75,49.631819,0.0068


,ALLFIVE,ASLAM,NurayBiasBorda30,NurayBiasCondorcet30,NurayBiasRankPosition30,NurayNormalBorda30,NurayNormalCondorcet30,NurayNormalRankPosition30,SAKAI30,SINGLE,SINGLEMINUSALLFIVE,SOBOROFF,WUCBASIC,WUCV1,WUCV2,WUCV3,WUCV4,AP
IND,,,,,,,,,,,,,,,,,,
CiirAll1_270,7.02,5.566484,0.3175,0.3706,0.1548,0.2753,0.2509,0.0511,0.3533,-32.28,-25.26,0.4107,8218,9571022,8194.79,9544908.69,-15.873025,0.0244
CiirAll1_271,4.06,2.983022,0.2564,0.2506,0.2564,0.2368,0.2042,0.0238,0.2619,-36.38,-32.32,0.2379,4460,4852856,4797.30,5283973.94,56.095417,0.0122
CiirAll1_272,5.50,2.544478,0.1821,0.1678,0.1755,0.2016,0.1740,0.0686,0.2166,-27.30,-21.80,0.1734,7281,7579356,7405.42,7655418.97,10.680147,0.1789
CiirAll1_273,7.08,4.853028,0.4525,0.4279,0.4525,0.4183,0.3621,0.0527,0.5141,-29.68,-22.60,0.4885,8298,10016855,8174.61,9893101.71,-37.820688,0.0736
CiirAll1_274,6.92,2.931164,0.1994,0.2117,0.1994,0.2277,0.1725,0.0376,0.2228,-32.14,-25.22,0.2177,7184,8043829,7168.12,8027687.99,-20.821750,0.1908


AssertionError: 

In [ ]:
## merge

In [ ]:
# tl

In [ ]:
for i, (collection_eval, others_coll) in enumerate(arg_instances):
    #print(collection_eval, others_coll)
    sub = pd.read_pickle('./pickles/partial/train{}-eval{}-algDNN.pickle'.format(others_coll, collection_eval))
    
    if i==0:
        dfres = pd.DataFrame(columns=sub.columns)
        
    dfres = pd.concat([dfres,sub])
    #display(sub)
    
dfres.to_pickle('./pickles/DNN.pickle')
display(dfres)

In [ ]:
# not tl

In [10]:
for i, (collection_eval, others_coll) in enumerate(arg_instances):
    #print(collection_eval, others_coll)
    sub = pd.read_pickle('./pickles/partial/train{}-eval{}-algDNN.pickle'.format(len(others_coll), collection_eval))
    
    
    if i==0:
        dfres = pd.DataFrame(columns=sub.columns)
        
    dfres = pd.concat([dfres,sub])
    #display(sub)
    
dfres.to_pickle('./pickles/DNN.pickle')
display(dfres)

,train_on,eval_on,improve_kind,kind,df
0,14,WEB14gd,Base,DNN,actual predicted ...
0,13,WEB13,Base,DNN,actual predicted ...
0,12,WEB12,Base,DNN,actual predicted syste...
0,11,WEB11,Base,DNN,actual predicted system ...
0,10,TB06,Base,DNN,actual predicted sys...
0,8,TB05,Base,DNN,actual predicted system ...
0,8,R05,Base,DNN,actual predicted system topi...
0,6,TB04,Base,DNN,actual predicted syste...
0,6,R04,Base,DNN,actual predicted system topic ...
0,5,TREC2001,Base,DNN,actual predicted system topi...


In [18]:
a = pd.read_pickle('./_pickles/partial/train1-evalTREC5-algk-BAYRIDGE.pickle')
display(a)

display( a.iloc[0]['df']  ) 

,train_on,eval_on,improve_kind,kind,df
0,1,TREC5,Base,k-BAYRIDGE,actual predicted system topic CL...


,actual,predicted,system,topic
CLCLUS_270,0.1008,0.073248,CLCLUS,270
CLCLUS_271,0.3778,0.350907,CLCLUS,271
CLCLUS_272,0.2626,0.389236,CLCLUS,272
CLCLUS_273,0.5590,0.263443,CLCLUS,273
CLCLUS_274,0.6537,0.266620,CLCLUS,274
CLCLUS_275,0.0109,0.246954,CLCLUS,275
CLCLUS_276,0.7509,0.341829,CLCLUS,276
CLCLUS_277,0.1036,0.104023,CLCLUS,277
CLCLUS_278,0.0863,0.149849,CLCLUS,278
CLCLUS_279,0.2692,0.074705,CLCLUS,279
